In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition


Traceback (most recent call last):
  File "c:\users\user\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\user\appdata\local\programs\python\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\User\AppData\Local\Programs\Python\Python37\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\kaggle\__init__.py", line 23, in <module>
    api.authenticate()
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\User\.kaggle. Or use the environment method.


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 809M/814M [00:06<00:00, 233MB/s]
100% 814M/814M [00:06<00:00, 133MB/s]


In [ ]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [ ]:
!unzip -q train.zip -d .

In [ ]:
# 파일 개수
import os
print(len(os.listdir('/content/train/')))

In [ ]:
!rm -r '/content/dataset/'

In [ ]:
!mkdir '/content/dataset/'
!mkdir '/content/dataset/cat/'
!mkdir '/content/dataset/dog/'

In [ ]:
import shutil
 
for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/'+i,'/content/dataset/cat/'+i)
  if 'dog' in i:
    shutil.copyfile('/content/train/'+i,'/content/dataset/dog/'+i)

In [ ]:
import tensorflow as tf
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(150,150),
    batch_size=64,
    subset="training",
    validation_split=0.2,
    seed=1234
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(150,150),
    batch_size=64,
    subset="validation",
    validation_split=0.2,
    seed=1234
)

def preproFun(i,result):
  i = tf.cast( i/255.0, tf.float32 )
  return i,result

train_ds = train_ds.map(preproFun)
val_ds = val_ds.map(preproFun)
import matplotlib.pyplot as plt

# for i,정답 in train_ds.take(1):
#   print(i)
#   print(정답)
#   plt.imshow(i[0].numpy().astype('uint8'))
#   plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
model = tf.keras.Sequential([
    
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64,64,3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),

    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds, epochs=5)

In [ ]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3(input_shape = (150,150,3),include_top=False,weights=None)
inception_model.load_weights('inception_v3.h5')

# inception_model.summary()

for i in inception_model.layers:
  i.trainable=False # w값 고정

unfreeze = False
for i in inception_model.layers:
  if i.name == 'mixed6':
    unfreeze = True
  if unfreeze == True :
    i.trainable = True

lastLayer = inception_model.get_layer('mixed7')




In [ ]:
import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(lastLayer.output)
layer2 = tf.keras.layers.Dense(1024,activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
output = tf.keras.layers.Dense(1,activation='sigmoid')(drop1)

model = tf.keras.Model(inception_model.input,output)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])
model.fit(train_ds,validation_data=val_ds, epochs=5)


Epoch 1/5
313/313 [==============================] - 61s 132ms/step - loss: 0.1472 - acc: 0.9548 - val_loss: 0.0944 - val_acc: 0.9696
Epoch 2/5
313/313 [==============================] - 38s 119ms/step - loss: 0.0447 - acc: 0.9830 - val_loss: 0.0773 - val_acc: 0.9724
Epoch 3/5
313/313 [==============================] - 34s 106ms/step - loss: 0.0245 - acc: 0.9915 - val_loss: 0.1022 - val_acc: 0.9690
Epoch 4/5
313/313 [==============================] - 38s 119ms/step - loss: 0.0150 - acc: 0.9949 - val_loss: 0.0901 - val_acc: 0.9740
Epoch 5/5
313/313 [==============================] - 38s 119ms/step - loss: 0.0143 - acc: 0.9951 - val_loss: 0.1937 - val_acc: 0.9638


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])
model.fit(train_ds,validation_data=val_ds, epochs=5)